In [1]:
import re

In [11]:
def extract_resource_usage(slurm_script):
    with open(slurm_script, 'r') as file:
        content = file.read()

    gpu_match = re.search(r"#SBATCH --gres=gpu:(\d+)", content)
    num_gpus = int(gpu_match.group(1)) if gpu_match else 0

    cpu_match = re.search(r"#SBATCH --cpus-per-task=(\d+)", content)
    if cpu_match:
        num_cpus = int(cpu_match.group(1))
    else:
        cpu_match = re.search(r"#SBATCH --cpus-per-gpu=(\d+)", content)
        num_cpus = int(cpu_match.group(1)) * num_gpus if cpu_match else 0

    memory_match = re.search(r"#SBATCH --mem=(\d+)", content)
    if memory_match:
        memory_gb = int(memory_match.group(1))
    else:
        memory_match = re.search(r"#SBATCH --mem-per-cpu=(\d+)", content)
        memory_gb = int(memory_match.group(1)) * num_cpus if memory_match else 0

    time_match = re.search(r"#SBATCH --time=(\d+)-(\d+):(\d+):(\d+)", content)
    if time_match:
        days, hours, minutes, seconds = map(int, time_match.groups())
        total_hours = days * 24 + hours + minutes / 60 + seconds / 3600
    else:
        time_match = re.search(r"#SBATCH --time=(\d+):(\d+):(\d+)", content)
        if time_match:
            hours, minutes, seconds = map(int, time_match.groups())
            total_hours = hours + minutes / 60 + seconds / 3600
        else:
            total_hours = 0

    # If using array
    array_match = re.search(r"#SBATCH --array=(\d+)-(\d+)", content)
    if array_match:
        array_start, array_end = map(int, array_match.groups())
        array_size = array_end - array_start + 1
    else:
        array_size = 1

    return num_gpus, num_cpus, memory_gb, total_hours, array_size

def calculate_cost(num_gpus, num_cpus, memory_gb, total_hours, array_size):
    # prices defined in Anunna wiki, in Euros
    gpu_cost_per_hour = 0.3000
    cpu_cost_per_hour = 0.0150
    memory_cost_per_hour = 0.0015

    total_cost = (
        (num_gpus * gpu_cost_per_hour +
         num_cpus * cpu_cost_per_hour +
         memory_gb * memory_cost_per_hour) *
        total_hours *
        array_size
    )

    return total_cost


Estimated Resource Usage Price: €23.0400


In [19]:
# Specify script name here and upload the script used
script_name = 'train_cropgym_array_rew.sh'
slurm_script = f'/content/{script_name}'


num_gpus, num_cpus, memory_gb, total_hours, array_size = extract_resource_usage(slurm_script)
total_cost = calculate_cost(num_gpus, num_cpus, memory_gb, total_hours, array_size)

print(
    f"Num gpus: {num_gpus}\n"
    f"Num cpus: {num_cpus}\n"
    f"Memory used: {memory_gb} GBs\n"
    f"Totak hours scheduled per-task: {total_hours} (may finish earlier, thus lower actual cost than estimated)\n"
    f"Job array size: {array_size}\n"
    f"Estimated Resource Usage Price: €{total_cost:.4f}"
  )

Num gpus: 0
Num cpus: 2
Memory used: 12 GBs
Totak hours scheduled per-task: 48.0 (may finish earlier, thus lower actual cost than estimated)
Job array size: 10
Estimated Resource Usage Price: €23.0400
